In [1]:
import pandas as pd
import re
import numpy as np
import time
import datetime

In [60]:
review_1 = pd.read_csv("1.csv")
review_2 = pd.read_csv("2.csv")
review_3 = pd.read_csv("3.csv")
review_4 = pd.read_csv("4.csv")

na_index = list(review_1[review_1['timestamp'].isna()].index)
for i in na_index:
    tmp = review_1.iloc[(i)][['itemid', 'userid']]
    review_1.loc[(i-1),'comment'] = review_1.loc[(i-1)]['comment'] + '\n' + tmp[0]
    review_1.loc[(i-1),'option'] = tmp[1]

review_1.dropna(axis=0, subset=['timestamp'], inplace=True)
review_1.dropna(axis=0, subset=['comment'], inplace=True)

na_index = list(review_3[review_3['timestamp'].isna()].index)
i = na_index[0]
tmp = review_3.iloc[(i)][['Unnamed: 0', 'itemid']]
review_3.loc[(i-1),'comment'] = review_3.loc[(i-1)]['comment'] + '\n' + tmp[0]
review_3.loc[(i-1),'option'] = tmp[1]

i = na_index[1]
tmp = review_3.iloc[i:(i+2),:][['Unnamed: 0', 'itemid']]
review_3.loc[(i-1),'comment'] = review_3.loc[(i-1)]['comment'] + '\n' + tmp.iloc[0][0] + '\n' + tmp.iloc[1][0]
review_3.loc[(i-1),'option'] = tmp.iloc[1][1]

review_1.dropna(subset='comment', inplace=True)
review_1['comment'] = review_1['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_2.dropna(subset='review', inplace=True)
review_2['review'] = review_2['review'].apply(lambda x: re.sub(r'\n',' ',x))

review_3.dropna(subset='comment', inplace=True)
review_3['comment'] = review_3['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_4.dropna(subset='comment', inplace=True)
review_4['comment'] = review_4['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_3.dropna(axis=0, subset=['userid'], inplace=True)
review_3.dropna(axis=0, subset=['comment'], inplace=True)
review_2.dropna(axis=0, subset=['review'], inplace=True)
review_4.dropna(axis=0, subset=['comment'], inplace=True)
review_3.drop('Unnamed: 0', axis=1, inplace=True)

review_2.columns = review_1.columns
review_total = pd.concat([review_1, review_2, review_3, review_4], axis=0)
review_total.columns = ['item_id', 'user_id', 'time', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'review', 'option']
review_total.reset_index(drop=True, inplace=True)

review_total['item_id'] = review_total['item_id'].apply(int)
review_total['user_id'] = review_total['user_id'].apply(int)
review_total['year'] = review_total['time'].apply(lambda x: int(x[:4]))
review_total['month'] = review_total['time'].apply(lambda x: int(x[5:7]))
review_total['day'] = review_total['time'].apply(lambda x: int(x[8:]))
review_total['review'] = review_total['review'].apply(lambda x: re.sub(r'\n',' ',x))

condition = (review_total['star_avg']>0) & (review_total['star_durability']==0) & (review_total['star_design']==0) & (review_total['star_cost']==0) & (review_total['star_delivery']==0)
idxs = review_total.loc[condition].index
review_total.iloc[idxs, 4] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 5] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 6] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 7] = review_total.iloc[idxs, 3]
review_total.loc[review_total['option'].isna(), ['option']] = 'NO'

/tmp/ipykernel_81706/1988394935.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_1 = pd.read_csv("1.csv")
/tmp/ipykernel_81706/1988394935.py:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_3 = pd.read_csv("3.csv")


In [64]:
review_total.to_csv("review_total.csv", index=False)

# inter

In [2]:
tmp = pd.read_csv("review_total.csv", lineterminator='\n')
inter_recbole = tmp.copy()
inter_recbole['time'] = inter_recbole['time'].apply(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%Y.%m.%d").timetuple())))
inter_recbole.head(3)

,item_id,user_id,time,star_avg,star_durability,star_design,star_cost,star_delivery,review,option,year,month,day
0,118895,7001668,1608595200,5.0,5.0,5.0,5.0,5.0,편안하고 깔끔하구요 살짝 좌식 시 미끄러운감은 있지만 사용에 문제없어요,NO,2020,12,22
1,118895,5813881,1587859200,3.0,3.0,3.0,3.0,3.0,화이트는 좀 관리가 힘든거 같아요... 배송좀신경써주셨으면...,NO,2020,4,26
2,118895,6447991,1583366400,5.0,5.0,5.0,5.0,5.0,아직 식탁이 안와서 의자사진만 있지만 튼튼하고 지정일 맞춰서 보내주셨어요~!,NO,2020,3,5


In [3]:
inter_recbole.columns = ['item_id', 'user_id', 'timestamp', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'review', 'option', 'year', 'month', 'day']
inter_recbole.drop(['year', 'month', 'day'], axis=1, inplace=True)

In [4]:
inter_recbole.columns = ['item_id:token', 'user_id:token', 'timestamp:float', 'star_avg:float', 'star_durability:float', 'star_design:float', 'star_cost:float', 'star_delivery:float', 'review:token_seq', 'option:token_seq']
inter_recbole.to_csv("final.csv", index=False)

# item

In [90]:
side = pd.read_csv("side_info.csv")

,review_avg,is_sold_out,is_selling,is_buyable,category0,category1,delivery_fee,delivery_fee_free_threshold,img_main,img_sub_list,option_names,max_option_price
0,4.560606,False,True,True,가구,거실장·TV장,30000,0,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['색상:클래식화이트', '색상:램프블랙']",469000
1,4.777778,False,True,True,주방용품,커피·티용품,3000,10000,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,['선택:600ml 스텐 우유 스팀피쳐/눈금형'],9900
2,4.646538,False,True,True,수납·정리,현관·신발정리,0,0,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['색상:화이트', '색상:블랙']",8900


In [99]:
side_recbole = side.drop(['is_selling', 'is_buyable', 'img_main', 'img_sub_list', 'option_names'], axis=1)
side_recbole['is_sold_out'] = side_recbole['is_sold_out'].apply(int)
side_recbole.columns = ['item_id:token', 'title:token_seq', 'brand:token', 'original_price:float', 'selling_price:float',
       'wish_count:float', 'share_count:float', 'scrap_count:float', 'qa_count:float', 'review_count:float',
       'review_avg:float', 'is_sold_out:float', 'category0:token', 'category1:token', 'delivery_fee:float', 'delivery_fee_free_threshold:float',
       'max_option_price:float']
side_recbole.head(5)

,item_id:token,title:token_seq,brand:token,original_price:float,selling_price:float,wish_count:float,share_count:float,scrap_count:float,qa_count:float,review_count:float,review_avg:float,is_sold_out:float,category0:token,category1:token,delivery_fee:float,delivery_fee_free_threshold:float,max_option_price:float
33170,449,[한정수량] LACK 벽 선반 110cm,이케아,30300,30300,23181,1900,23181,133,475,4.589474,1,가구,선반,3000,0,30300
38032,611,STROMBY 액자 2colors 5sizes,이케아,10900,10900,912,81,912,3,5,4.800000,1,데코·식물,홈갤러리,0,0,30900
17071,684,Mulig 벽 부착 옷걸이 행거,이케아,9900,9900,29938,3183,29938,95,652,4.578221,1,가구,행거·옷장,3000,0,9900
49299,2519,NISSE 접이식 의자 5colors,이케아,25900,25900,3093,227,3093,12,101,4.663366,1,가구,의자,0,0,25900
1180,3445,MKB 캐비넷 가로 2칸 TV스탠드 1180600 7colors,마켓비,127000,90000,17559,2235,17559,414,1241,4.350524,0,가구,서랍·수납장,20000,0,90000


In [97]:
side_recbole.to_csv("final.csv", index=False)

# user